# 生成試す

In [1]:
import os, sys, time, json, requests, re, random, string, argparse, logging, copy, glob, pathlib, shutil
import hashlib
import numpy as np
import pandas as pd
# import japanize_matplotlib
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.image as mpimg
%matplotlib inline
# import xml.etre.ElementTree as ET
from xml.dom import minidom
from lxml import etree

font_path = '/System/Library/Fonts/ヒラギノ角ゴシック W4.ttc'
font_prop = fm.FontProperties(fname=font_path)
plt.rcParams['font.family'] = font_prop.get_name()

## 画像作成

In [ ]:
df = pd.read_excel("data/test-play.xlsx", sheet_name="cardsv3")
df.head()

,id,level,res1,res2,1to2,engine,1toE,icon1,1toI1,icon2,1toI2,recycle,steps,score,timing,effect
0,NaN,1,1,1.0,1.0,1,2.0,1,0.0,NaN,NaN,1,1,1,NaN,NaN
1,NaN,1,1,1.0,1.0,1,3.0,1,1.0,NaN,NaN,1,1,1,NaN,NaN
2,NaN,1,1,1.0,1.0,1,4.0,1,2.0,NaN,NaN,1,1,1,NaN,NaN
3,NaN,1,1,1.0,2.0,1,1.0,1,3.0,NaN,NaN,1,1,1,NaN,NaN
4,NaN,1,1,1.0,2.0,1,3.0,1,4.0,NaN,NaN,1,1,1,NaN,NaN


In [ ]:
# cardsv2
save_dir = "img/face"
materials_dir = "materials"
if os.path.exists(save_dir):
    shutil.rmtree(save_dir)
os.makedirs(save_dir, exist_ok=True)
color_list = ["red", "blue", "green", "yellow", "purple"]
icon_img_list = ["fire", "drop", "leaf", "sun", "moon"]
icon_img_list = [mpimg.imread(os.path.join(materials_dir, f"{i}.png")) for i in icon_img_list]
any_icon_img = mpimg.imread("materials/star.png")
any_color = "gray"
counter = 0
fs = 12
height = 3.50; width = 2.48
hv_ratio = width / height
box_width = 0.1
box_height = box_width * hv_ratio
box_margin = 0.02


for i, row in df.iterrows():
    level = row["level"]
    for j, res1_color in enumerate(color_list):
        # fig, ax = plt.subplots(figsize=(8.27, 11.69))
        fig, ax = plt.subplots(figsize=(width, height))
        ax.set_xlim(0, 1)
        ax.set_ylim(0, 1)

        ax.text(0, 1, "COST", horizontalalignment='left', verticalalignment='bottom', fontsize=fs)
        res_counter = 0
        for _ in range(row["res1"]):
            rect = patches.Rectangle((0.02 + res_counter * (box_width + box_margin), 0.9), box_width, box_height, facecolor=res1_color, edgecolor="black")
            ax.add_patch(rect)
            res_counter += 1
        n_res2 = row["res2"]
        if not np.isnan(n_res2):
            n_res2 = int(n_res2)
            res2_color_id = (int(row["1to2"]) + j) % len(color_list)
            res2_color = color_list[res2_color_id]
            for _ in range(n_res2):
                rect = patches.Rectangle((0.02 + res_counter * (box_width + box_margin), 0.9), box_width, box_height, facecolor=res2_color, edgecolor="black")
                ax.add_patch(rect)
                res_counter += 1

        ax.text(0, 0.11, "ENGINE", horizontalalignment='left', verticalalignment='bottom', fontsize=fs)
        n_engine = row["engine"]
        engine_color = None
        if isinstance(n_engine, str):
            if n_engine == "W":
                engine_color = any_color
                n_engine = 1
            elif n_engine == "W2":
                engine_color = any_color
                n_engine = 2
        if not np.isnan(n_engine):
            if engine_color is None:
                engine_color_id = (int(row["1toE"]) + j) % len(color_list)
                engine_color = color_list[engine_color_id]
                n_engine = int(n_engine)
            engine_counter = 0
            for _ in range(n_engine):
                rect = patches.Rectangle((0.02 + engine_counter * (box_width + box_margin), 0.01), box_width, box_height, facecolor=engine_color, edgecolor="black")
                ax.add_patch(rect)
                engine_counter += 1

        ax.text(0, 0.75, "EFFECT", horizontalalignment='left', verticalalignment='bottom', fontsize=fs)
        effect = row["effect"]
        if not pd.isna(effect):
            ax.text(0, 0.65, effect, horizontalalignment='left', verticalalignment='bottom', fontsize=fs)

        ax.text(1, 0.75, "RECYCLE", horizontalalignment='right', verticalalignment='bottom', fontsize=fs)
        n_recycle = row["recycle"]
        if level == 1:
            if not n_recycle.isdigit():
                recycle_color = res1_color
            else:
                recycle_color = engine_color
        else:
            if n_recycle == "W":
                recycle_color = any_color
                n_recycle = 1
            else:
                recycle_color = engine_color
                n_recycle = int(n_recycle)
        recycle_counter = 1
        for _ in range(n_recycle):
            rect = patches.Rectangle((0.98 - recycle_counter * (box_width + box_margin), 0.65), box_width, box_height, facecolor=recycle_color, edgecolor="black")
            ax.add_patch(rect)
            recycle_counter += 1

        ax.text(1, 0.36, "ICONS", horizontalalignment='right', verticalalignment='bottom', fontsize=fs)
        icon_counter = 0
        n_icon1 = row["icon1"]
        icon_img = None
        if n_icon1 == "W":
            icon_img = any_icon_img
            n_icon1 = 1
        elif n_icon1 == "W2":
            icon_img = any_icon_img
            n_icon1 = 2
        if not np.isnan(n_icon1):
            if icon_img is None:
                icon_id = (int(row["1toI1"]) + j) % len(icon_img_list)
                icon_img = icon_img_list[icon_id]
                n_icon1 = int(n_icon1)
            for _ in range(n_icon1):
                ax.imshow(icon_img, extent=(0.98 - (icon_counter+1) * (box_width + box_margin), 0.98 - icon_counter * (box_width + box_margin), 0.26, 0.36), aspect="auto")
                icon_counter += 1
        
        n_icon2 = row["icon2"]
        if not np.isnan(n_icon2):
            icon_id = (int(row["1toI2"]) + j) % len(icon_img_list)
            icon_img = icon_img_list[icon_id]
            n_icon2 = int(n_icon2)
            for _ in range(n_icon2):
                ax.imshow(icon_img, extent=(0.98 - (icon_counter+1) * (box_width + box_margin), 0.98 - icon_counter * (box_width + box_margin), 0.26, 0.36), aspect="auto")
                icon_counter += 1

        step = int(row["steps"])
        ax.text(1, 1, f"STEP {step}", horizontalalignment='right', verticalalignment='bottom', fontsize=fs)

        ax.text(1, 0.11, "SCORE", horizontalalignment='right', verticalalignment='bottom', fontsize=fs)
        score = row["score"]
        if not np.isnan(score):
            score = int(score)
            ax.text(1, 0.01, score, horizontalalignment='right', verticalalignment='bottom', fontsize=fs)
        

        # ax.text(0.5, 0.5, row["effect"], horizontalalignment='center', verticalalignment='center', fontsize=20)
        ax.axis("off")
        fig.savefig(os.path.join(save_dir, f"{counter}.png"), dpi=300)
        counter += 1

        plt.close(fig)

In [ ]:
save_dir = "imgs/back"
text_list = ["世代1", "世代2", "世代3", "目標", "戦争"]
for i, t in enumerate(text_list):
    fig, ax = plt.subplots(figsize=(width, height))
    rect = patches.Rectangle((0, 0), 1, 1, facecolor="w", edgecolor="black")
    ax.text(0.5, 0.8, t, fontsize=32, ha="center", va="center")
    ax.axis("off")
    fig.savefig(os.path.join(save_dir, f"b{i+1}.png"), dpi=300)
    plt.close(fig)

In [ ]:
obj_save_dir = "imgs/obj/"
if os.path.exists(obj_save_dir):
    shutil.rmtree(obj_save_dir)
os.makedirs(obj_save_dir, exist_ok=True)
objective_list = [
    "5点/未保有の資源",
    "4点/未保有のアイコン",
    "3点/3以上保有の資源",
    "3点/3以上保有のアイコン",
    "5点/資源5種セット",
    "5点/アイコン5種セット",
    "1点/手札枚数",
    "7点"
]
objective_list += ["1枚:3点、2枚:10点、\n3枚:20点、4枚:32点" for _ in range(4)]
icon_img_name_list = ["fire", "drop", "leaf", "sun", "moon"]
icon_img_list = [mpimg.imread(f"imgs/{i}.png") for i in icon_img_name_list]
icon_jpn_name_list = ["炎", "水", "葉っぱ", "太陽", "月"]

for i, n in enumerate(icon_jpn_name_list):
    objective_list.append(f"2点/{n}")
    for j, n2 in enumerate(icon_jpn_name_list[i+1:]):
        objective_list.append(f"3点/({n},{n2})ペア")

height = 3.50; width = 2.48
fs = 12
for i, obj in enumerate(objective_list):
    fig, ax = plt.subplots(figsize=(width, height))
    ax.axis("off")
    ax.text(0.5, 0.5, obj, horizontalalignment='center', verticalalignment='center', fontsize=fs)
    fig.savefig(os.path.join(obj_save_dir, f"obj{i}.png"), dpi=300)
    plt.close(fig)

In [ ]:
war_save_dir = "imgs/war/"
if os.path.exists(war_save_dir):
    shutil.rmtree(obj_save_dir)
os.makedirs(war_save_dir, exist_ok=True)
war_list = []
war_jpn_name_list = ["赤", "青", "緑", "黄", "紫"]

for i, n in enumerate(war_jpn_name_list):
    war_list.append(f"{n} 多い方:5点")

height = 3.50; width = 2.48
fs = 12
for i, obj in enumerate(war_list):
    fig, ax = plt.subplots(figsize=(width, height))
    ax.axis("off")
    ax.text(0.5, 0.5, obj, horizontalalignment='center', verticalalignment='center', fontsize=fs)
    fig.savefig(os.path.join(war_save_dir, f"war{i}.png"), dpi=300)
    plt.close(fig)

## XML生成

In [ ]:
df = pd.read_excel("data/test-play.xlsx", sheet_name="cardsv2")
df.head()

,id,level,res1,res2,1to2,engine,1toE,icon1,1toI1,icon2,1toI2,recycle,steps,score,timing,effect
0,NaN,1,1,1.0,1.0,1,2.0,1,0.0,NaN,NaN,1,1,NaN,NaN,NaN
1,NaN,1,1,1.0,1.0,1,3.0,1,1.0,NaN,NaN,1,1,NaN,NaN,NaN
2,NaN,1,1,1.0,1.0,1,4.0,1,2.0,NaN,NaN,1,1,NaN,NaN,NaN
3,NaN,1,1,1.0,2.0,1,1.0,1,3.0,NaN,NaN,1,1,NaN,NaN,NaN
4,NaN,1,1,1.0,2.0,1,3.0,1,4.0,NaN,NaN,1,1,NaN,NaN,NaN


In [4]:
bucket_url = "https://r2-image-uploader.danjiri.workers.dev/"

# ルート要素を作成
def generate_root():
    root = etree.Element("card-stack", 
                    attrib={
                        "location.name": "table",
                        "location.x": "825",
                        "location.y": "1200",
                        "poxZ": "0",
                        "rotate": "0",
                        "zindex": "167",
                        "owner": "",
                        "isShowTotal": "true"
                        })
    card_stack = etree.SubElement(root, "data", attrib={"name": "card-stack"})
    image = etree.SubElement(card_stack, "data", attrib={"name": "image"})
    etree.SubElement(image, "data", attrib={"type": "image", "name": "imageIdentifier"}).text = ""
    common = etree.SubElement(card_stack, "data", attrib={"name": "common"})
    etree.SubElement(common, "data", attrib={"name": "name"}).text = "cards_stack"
    etree.SubElement(card_stack, "data", attrib={"name": "detail"}).text = ""
    return root


def create_card(card_root, name, face, back):
    card = etree.SubElement(card_root, "card", attrib={
        "location.name": "table",
        "location.x": "875",
        "location.y": "1525",
        "poxZ": "0",
        "state": "0",
        "rotate": "0",
        "owner": "",
        "zindex": "0"
        })
    card_data = etree.SubElement(card, "data", attrib={"name": "card"})
    image_data = etree.SubElement(card_data, "data", attrib={"name": "image"})
    etree.SubElement(image_data, "data", attrib={"type": "image", "name": "imageIdentifier"}).text = name
    etree.SubElement(image_data, "data", attrib={"type": "image", "name": "front"}).text = bucket_url + face
    etree.SubElement(image_data, "data", attrib={"type": "image", "name": "back"}).text = bucket_url + back
    common_data = etree.SubElement(card_data, "data", attrib={"name": "common"})
    etree.SubElement(common_data, "data", attrib={"name": "name"}).text = "カード"
    etree.SubElement(common_data, "data", attrib={"name": "size"}).text = "2"
    etree.SubElement(card_data, "data", attrib={"name": "detail"}).text = ""

for level in range(1, 4):
    current_df = df[df["level"] == level]
    root = generate_root()
    card_root = etree.SubElement(root, "node", attrib={"name": "cardRoot"})
    for k, row in current_df.iterrows():
        # level = row["level"]
        for j in range(5):
            i = 5*k + j
            create_card(card_root, str(i), f"{i}.png", f"b{level}.png")

    # 文字列に変換して整形
    xml_str = etree.tostring(root, encoding="utf-8", pretty_print=True, xml_declaration=True)

    # ファイル保存
    with open(os.path.join(f"data{level}.xml"), "wb") as f:
        f.write(xml_str)

In [ ]:
bucket_url = "https://r2-image-uploader.danjiri.workers.dev/"

root = generate_root()
card_root = etree.SubElement(root, "node", attrib={"name": "cardRoot"})
for i, obj in enumerate(objective_list):
    create_card(card_root, str(i), f"obj{i}.png", f"b4.png")

# 文字列に変換して整形
xml_str = etree.tostring(root, encoding="utf-8", pretty_print=True, xml_declaration=True)

# ファイル保存
with open(os.path.join("obj.xml"), "wb") as f:
    f.write(xml_str)

In [8]:
bucket_url = "https://r2-image-uploader.danjiri.workers.dev/"

root = generate_root()
card_root = etree.SubElement(root, "node", attrib={"name": "cardRoot"})
for i, war in enumerate(war_jpn_name_list):
    create_card(card_root, str(i), f"war{i}.png", f"b5.png")

# 文字列に変換して整形
xml_str = etree.tostring(root, encoding="utf-8", pretty_print=True, xml_declaration=True)

# ファイル保存
with open(os.path.join("war.xml"), "wb") as f:
    f.write(xml_str)